## ATDome failure test

In [1]:
import sys, time, os, asyncio

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient


Bad key "text.kerning_factor" on line 4 in
/opt/lsst/software/stack/conda/miniconda3-py37_4.8.2/envs/lsst-scipipe-cb4e2dc/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [3]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [4]:
#get classes and start them
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.11 sec
Read 3 history items for RemoteEvent(ATSpectrograph, 0, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(ATSpectrograph, 0, authList)
Read 12 history items for RemoteEvent(ATSpectrograph, 0, disperserInPosition)
Read 12 history items for RemoteEvent(ATSpectrograph, 0, filterInPosition)
Read 9 history items for RemoteEvent(ATSpectrograph, 0, fwState)
Read 12 history items for RemoteEvent(ATSpectrograph, 0, gwState)
Read 100 history items for RemoteEvent(ATSpectrograph, 0, heartbeat)
Read 1 history items for RemoteEvent(ATSpectrograph, 0, logLevel)
Read 1 history items for Remot

[[None, None, None, None, None, None, None], [None, None, None, None]]

timeAndDate DDS read queue is full (100 elements); data may be lost


In [5]:
# enable components if required
await atcs.enable()
#await latiss.enable()

Enabling all components
Gathering settings.
Couldn't get settingVersions event. Using empty settings.
Complete settings for atmcs.
Complete settings for atptg.
Complete settings for ataos.
Complete settings for atpneumatics.
Complete settings for athexapod.
Complete settings for atdome.
Complete settings for atdometrajectory.
Settings versions: {'atmcs': '                                                                                                                               ', 'atptg': '', 'ataos': 'current', 'atpneumatics': '                                                                                                                               ', 'athexapod': 'summit', 'atdome': 'test', 'atdometrajectory': ''}
[atmcs]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atptg]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[ataos]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atpneumatics]::[<State.STANDBY: 5>, <State.DISABLED: 

In [ ]:
# All components now ENABLED

In [6]:
# take event checking out the slew commands to test telescope only
# otherwise it'll wait for the dome before completing slew command
atcs.check.atdome = False
atcs.check.atdometrajectory = False

In [7]:
# turn on ATAOS corrections just to make sure the mirror is under air
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=False)

In [8]:
# Ensure we're using Nasmyth 2
await atcs.rem.atptg.cmd_focusName.set_start(focus=3)

In [9]:
# Now shut down
await atcs.shutdown()

Disabling ATAOS corrections
Cover state <MirrorCoverState.CLOSED: 6>
M1 cover already closed.
M1 vent state <VentsPosition.CLOSED: 1>
M1 vents already closed.
Close dome.
ATDome Shutter Door is already closed. Ignoring.
Slew dome to Park position.


RuntimeError: ATDome is deactivated. Activate it by setting `check.atdome=True` before slewing.In some cases users deactivate a component on purpose.Make sure it is clear to operate the dome before doing so.

In [10]:
# Putting everything back in standby.
# Need to do it one at a time, since ATDome fails with check=False.
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.ataos, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.atpneumatics, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.STANDBY)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [12]:
await atcs.rem.atdome.cmd_start.set_start(settingsToApply="test", timeout=30)

In [13]:
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY, settingsToApply="test")

[<State.DISABLED: 1>, <State.STANDBY: 5>]